In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 

import pandas as pd 

In [2]:
browser = webdriver.Firefox() 
url = 'https://www.bbc.com/news/world/europe'
browser.get(url)
print(f"Successfully visited: {browser.title}")

Successfully visited: Europe | Latest News & Updates | BBC News


Removing Cookies 


In [4]:
browser.switch_to.frame(browser.find_element(By.CSS_SELECTOR, "iframe[id^='sp_message_iframe']"))

button1 = browser.find_element(By.CSS_SELECTOR, "button[title='I agree']")
button1.click()
browser.switch_to.default_content()


Changes Region-Section 


In [26]:
button2 = browser.find_element(By.LINK_TEXT , 'Europe')
button2.click()


In [ ]:

def scrape_via_next_button(driver, num_pages):
    wait = WebDriverWait(driver, 10)
    data = []
    for i in range(1, num_pages + 1):
        print(f"--- Processing Page {i} ---")
        # 1. SCRAPE THE CURRENT PAGE
        # (Your scraping logic goes here)
        links = browser.find_elements(By.TAG_NAME, 'a')
        for link in links:
            try:
                # 1. Try to find a headline INSIDE this link
                # (This will fail for menu buttons(headers,footers...), but work for article cards (trageted news "cards")
                hd = link.find_element(By.CSS_SELECTOR, "[data-testid='card-headline']").text
                summ= link.find_element(By.CSS_SELECTOR,"[data-testid='card-description']").text
                url = link.get_attribute('href')
                data.append  ({
                    'Headlines' : hd ,
                    'Summary' : summ,
                    'URL' : url})

                # print(f"Headline: {hd}")
                # print(f"Summary: {summ}")
                # print(f"URL:    {url}")
                # print("-" * 20)
                
            except:
                # If the link doesn't have a headline inside it, just skip it!
                continue    
        # 2. CLICK NEXT (Don't click on the very last page!)
        if i < num_pages:
            try:
                
                # Find the 'Next' button (replace with actual ID/Class/Text)
                next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,f"button[aria-label='Go to page {i+1}']")))
                
                # Scroll to it if necessary
                driver.execute_script("arguments[0].scrollIntoView();", next_btn)
                next_btn.click()
                
                # Wait for old content to disappear (optional but recommended)
                time.sleep(2)
                
            except Exception as e:
                print(f"Could not find next button: {e}")
                break
	# elif i>num_pages : 
	# 				try : 
	# 					btn3 = browser.find_element(By.LINK_TEXT , 'UK')
    
    df = pd.DataFrame(data) 
    return df
    



In [50]:
df = scrape_via_next_button(browser, 7)

--- Processing Page 1 ---
--- Processing Page 2 ---
--- Processing Page 3 ---
--- Processing Page 4 ---
--- Processing Page 5 ---
--- Processing Page 6 ---
--- Processing Page 7 ---


In [51]:
df 


,Headlines,Summary,URL
0,Wave of arrests over killing of French nationa...,Eleven people are in detention in connection w...,https://www.bbc.com/news/articles/c62dzgy0q37o
1,Food prices are surging in Russia. Is the war ...,Russia's economy is hanging between stagnation...,https://www.bbc.com/news/articles/c1mjrgkk1m8o
2,Third Briton dies in French Alps avalanches in...,A total of 28 people have died in avalanches s...,https://www.bbc.com/news/articles/cy57nxrr6dqo
3,'Difficult' Russia-Ukraine peace talks end wit...,The second day of negotiations between Ukraine...,https://www.bbc.com/news/articles/c0k1xj0d708o
4,Spain luxury hotel scammer booked rooms for on...,Police arrest a 20-year-old suspected of defra...,https://www.bbc.com/news/articles/c0q3nwdk315o
...,...,...,...
205,France murder trial complicated by twin brothe...,Police have recovered DNA from a weapon but ca...,https://www.bbc.com/news/articles/cn87l12e4xeo
206,Italian police arrest protesters after clash i...,At least six people have been detained a week ...,https://www.bbc.com/news/articles/cgl5293n5n9o
207,France investigates ex-minister Jack Lang over...,The socialist grandee quits his role at a pres...,https://www.bbc.com/news/articles/crkrk567d7zo
208,Man critical after being hit by car,The crash happened in the early hours of Satur...,https://www.bbc.com/news/articles/cr7j7d4r3rlo
